# Cryptocurrencies Indicators

## Import libraries

In [38]:
import pandas as pd
import numpy as np
import pandas_ta as pda
import ta
import requests
from binance.client import Client
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import mplfinance as mpf
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

## Create the Dataframe

In [39]:
# Variables
# -- You can change the crypto pair ,the start date and the time interval below --
client = Client()
pair_symbol = "BTCUSDT"
time_interval = Client.KLINE_INTERVAL_1DAY
start_date = "01 january 2016"

# Fetch data
klinesT = client.get_historical_klines(pair_symbol, time_interval, start_date)

# Create dataframe
df = pd.DataFrame(klinesT, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])

# Drop unnecessary columns
df.drop(['close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore'], axis=1, inplace=True)

# Convert columns to numeric
for col in df.columns:
    df[col] = pd.to_numeric(df[col])

# Convert dates to datetime
df = df.set_index(df['timestamp'])
df.index = pd.to_datetime(df.index, unit='ms')
del df['timestamp']

# Display df
df

,open,high,low,close,volume
timestamp,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060
...,...,...,...,...,...
2023-03-13,21998.05,24500.00,21813.88,24113.48,687889.312590
2023-03-14,24112.27,26386.87,23976.42,24670.41,699360.934230
2023-03-15,24670.41,25196.97,23896.95,24285.66,581450.729840


## Create indicators 

In [40]:
df.drop(columns = df.columns.difference(['timestamp','open','high','low','close','volume']), inplace=True)

# EMA
df['ema7']=ta.trend.ema_indicator(close=df['close'], window=7)
df['ema20']=ta.trend.ema_indicator(close=df['close'], window=20)
df['ema50']=ta.trend.ema_indicator(close=df['close'], window=50)
df['ema100']=ta.trend.ema_indicator(close=df['close'], window=100)
df['ema200']=ta.trend.ema_indicator(close=df['close'], window=200)
#------------------------- #

# Bollingers Bands
indicator_bb = ta.volatility.BollingerBands(close=df["close"], window=20, window_dev=2)
df['bb_upper'] = indicator_bb.bollinger_hband()
df['bb_middle'] = indicator_bb.bollinger_mavg()
df['bb_lower'] = indicator_bb.bollinger_lband()
# ------------------------- #

# MACD
macd = ta.trend.MACD(close=df['close'], window_fast=12, window_slow=26, window_sign=9)
df['macd'] = macd.macd()
df['macd_signal'] = macd.macd_signal()
df['macd_histo'] = macd.macd_diff() # MACD Chart
#----------------------- #

# RSI
df['rsi'] = ta.momentum.RSIIndicator(close=df['close'], window=14).rsi()
#------------------------- #

# STOCHASTIC RSI
df['stoch_rsi'] = ta.momentum.stochrsi(close=df['close'], window=14)
df['stochastic'] = ta.momentum.stoch(high=df['high'],low=df['low'],close=df['close'], window=14,smooth_window=3)
df['stoch_signal'] =ta.momentum.stoch_signal(high =df['high'],low=df['low'],close=df['close'], window=14, smooth_window=3)
#------------------------- #

# Awesome Oscillator
df['awesome_oscilllator'] = ta.momentum.awesome_oscillator(high=df['high'], low=df['low'], window1=5, window2=34)
#---------------------- #

# ADX
df['adx'] =ta.trend.adx(high=df['high'], low=df['low'], close = df['close'], window = 14)
#----------------------- #

# Fear and Greed 
def fear_and_greed(close):
    ''' Fear and greed indicator
    '''
    response = requests.get("https://api.alternative.me/fng/?limit=0&format=json")
    dataResponse = response.json()['data']
    fear = pd.DataFrame(dataResponse, columns = ['timestamp', 'value'])

    fear = fear.set_index(fear['timestamp'])
    fear.index = pd.to_datetime(fear.index, unit='s')
    del fear['timestamp']
    df = pd.DataFrame(close, columns = ['close'])
    df['fearResult'] = fear['value']
    df['FEAR'] = df['fearResult'].ffill()
    df['FEAR'] = df.FEAR.astype(float)
    return pd.Series(df['FEAR'], name="FEAR")
df["fear&greed"] = fear_and_greed(df["close"])
#----------------------- #

# Heiki Ashi Bar
def heikinashi_df(df):
    df['ha_close'] = (df.open + df.high + df.low + df.close)/4
    ha_open = [(df.open[0] + df.close[0]) / 2]
    [ha_open.append((ha_open[i] + df.ha_close.values[i]) / 2)
     for i in range(0, len(df)-1)]
    df['ha_open'] = ha_open
    df['ha_high'] = df[['ha_open', 'ha_close', 'high']].max(axis=1)
    df['ha_low'] = df[['ha_open', 'ha_close', 'low']].min(axis=1)
    return df
#------------------------- #
df = heikinashi_df(df)

df

,open,high,low,close,volume,ema7,ema20,ema50,ema100,ema200,...,stoch_rsi,stochastic,stoch_signal,awesome_oscilllator,adx,fear&greed,ha_close,ha_open,ha_high,ha_low
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,NaN,4308.1725,4273.280000,4485.390000,4200.740000
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,NaN,4175.9350,4290.726250,4371.520000,3938.770000
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,NaN,4070.7600,4233.330625,4233.330625,3850.000000
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,NaN,4112.7425,4152.045313,4211.080000,4032.620000
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,NaN,4029.1350,4132.393906,4132.393906,3911.790000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-13,21998.05,24500.00,21813.88,24113.48,687889.312590,22077.342758,22378.549157,22188.253029,21314.817286,21773.711145,...,1.000000,92.192950,55.854385,-1721.724118,22.684481,49.0,23106.3525,20885.604077,24500.000000,20885.604077
2023-03-14,24112.27,26386.87,23976.42,24670.41,699360.934230,22725.609569,22596.821618,22285.592518,21381.264666,21802.534019,...,1.000000,74.897408,74.030266,-935.599059,23.761502,56.0,24786.4925,21995.978288,26386.870000,21995.978288
2023-03-15,24670.41,25196.97,23896.95,24285.66,581450.729840,23115.622176,22757.663369,22364.026537,21438.777445,21827.241741,...,0.915212,69.270582,78.786980,-82.519500,24.678245,50.0,24512.4975,23391.235394,25196.970000,23391.235394


## Plot the charts with indicators

In [41]:
# Create the main figure with subplots
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.03)

# Add the candlestick chart
fig.add_trace(go.Candlestick(x=df.index, open=df['ha_open'], high=df['ha_high'], low=df['ha_low'], close=df['ha_close']), row=1, col=1)

# Add the RSI subplot
fig.add_trace(go.Scatter(x=df.index, y=df['rsi'], name='RSI'), row=2, col=1)

# Add the volume subplot
fig.add_trace(go.Bar(x=df.index, y=df['volume'], name='Volume'), row=3, col=1)

# Customize the layout
fig.update_layout(title='Heikin Ashi Bars with RSI and Volume',
                  yaxis=dict(title='Price'),
                  xaxis_rangeslider_visible=False,
                  height=900)

# Show the chart
fig.show()

In [42]:
# Create a candlestick chart
fig = go.Figure(data=go.Candlestick(x=df.index,
                open=df['ha_open'],
                high=df['ha_high'],
                low=df['ha_low'],
                close=df['ha_close']))

# Add Bollinger Bands
fig.add_trace(go.Scatter(x=df.index, y=df['bb_upper'], name='Upper Band'))
fig.add_trace(go.Scatter(x=df.index, y=df['bb_middle'], name='Middle Band'))
fig.add_trace(go.Scatter(x=df.index, y=df['bb_lower'], name='Lower Band'))

# Customize the chart layout
fig.update_layout(
    title='Heikin Ashi Bars with Bollinger Bands',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False)

# Show the chart
fig.show()

In [43]:
# Create a candlestick chart
fig = go.Figure(data=go.Candlestick(x=df.index,
                open=df['ha_open'],
                high=df['ha_high'],
                low=df['ha_low'],
                close=df['ha_close']))

# Add EMA's
fig.add_trace(go.Scatter(x=df.index, y=df['ema20'], name='EMA20', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df.index, y=df['ema50'], name='EMA50', line=dict(color='yellow')))

# Customize the chart layout
fig.update_layout(
    title='Heikin Ashi Bars with EMA20 and EMA50',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False)

# Show the chart
fig.show()